In [431]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import copy
import time

M = 10
N = 10
S = 10
K = 10
c = [1,2,3,4,5,6,7,8,9,10]
r = [1,2,3,4,5,6,7,8,9,10]
MUT_RATE = 0.05
POP = 100
GEN = 200
POP_fitness = []


In [432]:
# np array with 20 cell each cell has 3 element

def init(POP):
    pop = []
    for i in range(POP):
        s = np.zeros((S,3))
        for j in range(S):
            s[j][0] = random.randint(0, K)
            s[j][1] = random.randint(0, M)
            s[j][2] = random.randint(0, N)
        pop.append(s.astype(int))
    return pop


In [433]:
def calculate_covered_cells(chromosome):
    covered_cells = np.zeros((M+1,N+1))
    for i in range(M+1):
        for j in range(N+1):
            for k in range(S):
                if chromosome[k][0] != 0 and covered_cells[i][j] == 0:
                    try:
                        if math.sqrt((i - chromosome[k][1]) ** 2 + (j - chromosome[k][2]) ** 2) <= r[chromosome[k][0]-1]:
                            covered_cells[i][j] = 1
                    except:
                        print("error")
                        print(chromosome[k][0])
                        print(r[chromosome[k][0]-1])
                        print(math.sqrt((i - chromosome[k][1]) ** 2 + (j - chromosome[k][2]) ** 2))
                        print(i)
                        print(j)
                        print(k)
                        print(chromosome[k][1])
                        print(chromosome[k][2])
                        print(chromosome)
                        print(covered_cells)
                        print("error")
    # print(covered_cells)
    return covered_cells

In [434]:
# claculate number of same individual
def calculate_cost(choromosome):
    cost = 0
    for i in range(S):
         if choromosome[i][0] == 0:
            cost += 0
         else:
            cost += c[choromosome[i][0]-1]
   #  print(cost)
    return cost


In [435]:
# calculate the fitness of each individual
def calculate_fitness(chromosome):
    fitness = 0
    fitness += np.sum(calculate_covered_cells(chromosome))
    fitness -= calculate_cost(chromosome)
    # print(fitness)
    return fitness

In [436]:
def pop_fitness(population):
    fitness = []
    for i in range(len(population)):
        fitness.append(calculate_fitness(population[i]))
    return fitness

In [437]:
# 1 point crossover
def crossover1point(choromosome1, choromosome2):
    child1 = copy.deepcopy(choromosome1)
    child2 = copy.deepcopy(choromosome2)
    index = random.randint(1, S - 1)
    child1[index:] = choromosome2[index:]
    child2[index:] = choromosome1[index:]
    return child1, child2 



In [438]:
# 4 point crossover
def crossover4point(choromosome1, choromosome2):
    child1 = copy.deepcopy(choromosome1)
    child2 = copy.deepcopy(choromosome2)
    index = random.sample(range(1, S - 1), 4)
    index.sort()
    child1[index[0]:index[1]] = choromosome2[index[0]:index[1]]
    child1[index[2]:index[3]] = choromosome2[index[2]:index[3]]
    child2[index[0]:index[1]] = choromosome1[index[0]:index[1]]
    child2[index[2]:index[3]] = choromosome1[index[2]:index[3]]
    return child1, child2

In [439]:
# uniform crossover
def crossoveruniform(choromosome1, choromosome2):
    child1 = copy.deepcopy(choromosome1)
    child2 = copy.deepcopy(choromosome2)
    for i in range(S):
        if random.random() < 0.5:
            try:
                 child1[i] = choromosome2[i]
                 child2[i] = choromosome1[i]
            except:
                print("error")
                print(i)
                print(choromosome1)
                print(choromosome2)
                print(child1)
                print(child2)
                print("error")

    return child1, child2

In [440]:
# mutation function
def mutation(choromosome):
    for i in range(S) :
        index = random.randint(0, 2)
        if random.random() < MUT_RATE:
            if index == 0:
                choromosome[i][index] = random.randint(0, K)
            elif index == 1:
                choromosome[i][index] = random.randint(0, M)
            elif index == 2:
                choromosome[i][index] = random.randint(0, N)
        return choromosome


In [441]:

def tournament_selection(population, k):
    best = None
    fitness = POP_fitness
    for i in range(k):
        candidate = random.randint(0, len(population)-1)
        # print(candidate)
        if best is None or fitness[candidate] < fitness[best]:
            best = candidate
    return population[best]





In [442]:
# roulette wheel selection function
def roulette_wheel_selection(population):
    fitness = POP_fitness
    total_fitness = sum(fitness)
    r = random.uniform(0, total_fitness)
    # print(r)
    upto = 0
    for i in range(len(population)):
        if upto + fitness[i] >= r:
            return population[i]
        upto += fitness[i]
    


In [443]:
# SUS selection function
def SUS_selection(population):
    fitness = POP_fitness
    total_fitness = sum(fitness)
    r = random.uniform(0, total_fitness/len(population))
    upto = 0
    for i in range(len(population)):
        if upto + fitness[i] >= r:
            return population[i]
        upto += fitness[i]

In [444]:
# genetic algorithm with 1% elitism
def genetic_algorithm_1():
    population = init(POP)
    best_fitness = []
    mean_fitness = []
    best_parent = None
    best_parent_fitness = None
    global POP_fitness
    for i in range(GEN):
        POP_fitness = pop_fitness(population)
        if(i != 0):
            # print("best fitness: ", best_parent_fitness)
            population[POP_fitness.index(min(POP_fitness))] = best_parent
            POP_fitness[POP_fitness.index(min(POP_fitness))] = best_parent_fitness

        best_fitness.append(max(POP_fitness))
        mean_fitness.append(sum(POP_fitness)/len(POP_fitness))
        print("generation ", i+1 )  
        best_parent = population[POP_fitness.index(max(POP_fitness))]
        best_parent_fitness = max(POP_fitness)
        new_population = []
        for j in range(POP//2):

            parent1 = tournament_selection(population , 2000)
            parent2 = tournament_selection(population , 2000)
            child1, child2 = crossoveruniform(parent1, parent2)
            child1 = mutation(child1)
            child2 = mutation(child2)
            new_population.append(child1)
            new_population.append(child2)

        population = new_population

    return best_fitness , mean_fitness

In [445]:
# second genetic algorithm with 98% elitism
def genetic_algorithm_98():
    population = init(POP)
    best_fitness = []
    mean_fitness = []
    best_parent = None
    best_parent_fitness = None
    global POP_fitness
    for i in range(GEN):
        print("generation ", i+1 )
        
        POP_fitness = pop_fitness(population)
        best_fitness.append(max(POP_fitness))
        mean_fitness.append(sum(POP_fitness)/len(POP_fitness))

        best_parent = population[POP_fitness.index(max(POP_fitness))]
        best_parent_fitness = max(POP_fitness)

        parent1 = tournament_selection(population , 5)
        parent2 = tournament_selection(population , 5)
        child1, child2 = crossoveruniform(parent1, parent2)

        child1 = mutation(child1)
        child2 = mutation(child2)
        
        population[POP_fitness.index(min(POP_fitness))] = child1
        POP_fitness[POP_fitness.index(min(POP_fitness))] = calculate_fitness(child1)
        population[POP_fitness.index(min(POP_fitness))] = child2
        POP_fitness[POP_fitness.index(min(POP_fitness))] = calculate_fitness(child2)

            
    return best_fitness , mean_fitness

In [446]:
# main function
def main():

    best_fitness, mean_fitness = genetic_algorithm_98()
    # best_fitness, mean_fitness = genetic_algorithm_1()
    
    plt.plot(best_fitness)
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.title("Best Fitness")
    plt.show()
    # plot the mean fitness
    plt.plot(mean_fitness)
    plt.xlabel("Generation")
    plt.ylabel("Mean Fitness")
    plt.title("Mean Fitness")
    plt.show()


In [ ]:
if __name__ == "__main__":
    main()
